# Quantization of embeddings
In similarity search, one often needs to store and process thousands or even millions of high-dimensional embedding vectors. These vectors consume substantial memory, and as the dataset grows, search operations become increasingly slow. To address these challenges, we seek alternative representations of embedding vectors that reduce storage requirements and enhance computational efficiency.

## Packages
In this notebook, the following python packages will be used.

In [1]:
from collections.abc import Callable
import math
import random
import numpy as np

## Motivation
Quantization is a technique that helps to:
- Reduce memory usage by storing embeddings in a compressed form.
- Improve computational efficiency by enabling faster similarit calculations using lower-precision representations.

## Mathematical background
Let $\mathrm{min}, \mathrm{max}\in\mathbb{R}$ and $a, b\in\mathbb{Z}$. Suppose we have a value $x$ in the range $[\mathrm{min}, \min{max}]$, and we want to map it to an integer $\overline{x}$ in the discrete range $[a, b]\cap\mathbb{Z}$. We achieve this as follows: first we define the $\textbf{quantization step size}$ $\Delta = \frac{\mathrm{max} - \mathrm{min}}{b - a}$ and set

$$\tilde{x} = a + \frac{x - \mathrm{min}}{\Delta}\in [a,b].$$

Then we define $\overline{x}$ as the result of the of one of the rounding operators round, ceil or floor on $\tilde{x}$, e.g. 
$$\overline{x} = \operatorname{round}(\tilde{x}) = a  + \operatorname{round}\left(\frac{x-\mathrm{min}}{\Delta}\right).$$

In [2]:
min_v = -1
max_v = 1
a = -128
b = 127

delta = (max_v - min_v) / (b - a)

d = 5

In [3]:
def forward_transformation(x: float, transform: Callable[[float], int] = math.floor) -> int:
    x_tilde = a + (b - a) / (max_v - min_v) * (x - min_v)
    return transform(x_tilde)

The backward transformation is defined as $\hat{x} = (\overline{x} - a) \cdot \Delta + \mathrm{min}$.

In [4]:
def backward_transformation(x: int) -> float:
    return (x - a) * (max_v - min_v) / (b - a) + min_v

## Estimation of the quantization error
Due to the rounding operation in quantization, the reconstructed value $\hat{x}$ does not exactly match the original value $x$. This discrepancy is called the $\textbf{quantization error}$.
For flooring and ceiling operations, we can establish the bound:
$$
\vert x - \hat{x}\vert < \Delta.
$$
Flooring always results in a reconstructed value that is less than or equal to $x$, while ceiling results in a value that is greater than or equal to $x$.
For rounding, the quantization error can be improved to:
$$
\vert x - \hat{x}\vert \leq \frac{\Delta}{2}.
$$
<details open>
<summary>Proof</summary>

<b>Flooring</b>\
From the definition of $\overline{x}=a + \lfloor\frac{x - \mathrm{min}}{\Delta}\rfloor$ it follows that
$$
\overline{x} \leq a + \frac{x - \mathrm{min}}{\Delta} < \overline{x} + 1.
$$
Solving for $x$ we see that
$$
\mathrm{min} + (\overline{x} - a)\cdot\Delta \leq x < \mathrm{min} + (\overline{x} - a + 1)\cdot\Delta
$$
and hence by the definition of the backwards transformation it follows that
$$
\hat{x} \leq x < \hat{x} + \Delta.
$$
Consequently, it holds $\vert x - \hat{x}\vert < \Delta$.

<b>Ceiling</b>\
Analogously, we see from the definition $\overline{x} = a + \lceil \frac{x - \mathrm{min}}{\Delta}\rceil$ that
$$
\overline{x} - 1 < a + \frac{x - \mathrm{min}}{\Delta} \leq \overline{x}
$$
and therefore
$$
\mathrm{min} + (\overline{x} - a - 1)\cdot\Delta < x \leq \mathrm{min} + (\overline{x} - a)\cdot\Delta
$$
and hence
$$
\hat{x} - \Delta < x \leq \hat{x}.
$$
In other words, it holds $\vert x - \hat{x}\vert < \Delta$.

<b>Rounding</b>
Similarly, from the definition $\overline{x} = a + \operatorname{round}(\frac{x - \mathrm{min}}{\Delta})$ we see that
$$
\overline{x} - \frac{1}{2} \leq a + \frac{x - \mathrm{min}}{\Delta} \leq \overline{x} + \frac{1}{2}.
$$
Again, it follows that
$$
\mathrm{min} + \left(\overline{x} - a - \frac{1}{2}\right) \leq x \leq \mathrm{min} + \left(\overline{x} - a + \frac{1}{2}\right)
$$
and hence
$$
\hat{x} - \frac{1}{2} \leq x \leq \hat{x} + \frac{1}{2}.
$$
Therefore, it holds $\vert x - \hat{x}\vert \leq \frac{\Delta}{2}$. $\square$
</details>

<b>Dot product</b>\
Next, we want to expand this to dot products. For that, let $\epsilon_x\in\mathbb{R}$ denote the exact quantization error, i.e. in the case of flooring $x = \hat{x} + \epsilon_x$.

Now, let $x, y\in\mathbb{R}^d$ be two vectors. Then we can quantize each entry and reconstrunct the values $x$ and $y$. By doing this we obtain the two vectors $\hat{x}, \hat{y}\in\mathbb{R}^d$ with the property $(x_i)_{1\leq i\leq d} = (\hat{x}_i + \epsilon_{x_i})_{1\leq i\leq d}$ and $(y_i)_{1\leq i\leq d} = (\hat{y}_i + \epsilon_{y_i})_{1\leq i\leq d}$. Then the product of two entries can be written as
$$
x_i\cdot y_i = (\hat{x}_i + \epsilon_{x_i})(\hat{y_i} + \epsilon_{y_i}) = \hat{x}_i\hat{y}_i + \hat{x}_i\epsilon_{y_i} + \hat{y}_i\epsilon_{x_i} + \epsilon_{x_i}\epsilon_{y_i}.
$$
For the dot product we then can show that
$$
\begin{align*}
\vert\langle x, y\rangle - \langle\hat{x}, \hat{y}\rangle\vert &= \left\vert\sum_{i=1}^d(\hat{x}_i\epsilon_{y_i} + \hat{y}_i\epsilon_{x_i} + \epsilon_{x_i}\epsilon_{y_i})\right\vert\\
&\leq \sum_{i=1}^d\vert\hat{x}_i\epsilon_{y_i}\vert + \sum_{i=1}^d\vert\hat{y}_i\epsilon_{x_i}\vert + \sum_{i=1}^d \vert\epsilon_{x_i}\epsilon_{y_i}\vert\\
&\leq \Delta \sum_{i=1}^d\vert\hat{x}_i\vert + \Delta\sum_{i=1}^d\vert\hat{y}_i\vert + d\Delta^2\\
&= \Delta(\lVert\hat{x}\rVert_1 + \lVert\hat{y}\rVert_1) + d\Delta^2\\
&\leq \Delta(\lVert x\rVert_1 + \lVert y\rVert_1) + d\Delta^2\\
&\leq \Delta (\sqrt{d}\lVert x \rVert_2 + \sqrt{d}\lVert y \rVert_2) + d\Delta^2\\
&= \sqrt{d}\Delta (\lVert x \rVert_2 + \lVert y \rVert_2) + d\Delta^2.
\end{align*}
$$
In most cases, embedding vectors are normalized, i.e. $\lVert x\rVert_2 = \lVert y\rVert_2 = 1$. Therefore, we can bound the error to
$$
\vert\langle x, y\rangle - \langle\hat{x}, \hat{y}\rangle\vert \leq 2\sqrt{d}\Delta + d\Delta^2.
$$

If we use ceiling instead, we get the same upper bound. If we use rounding, we can improve it to
$$
\vert\langle x, y\rangle - \langle\hat{x}, \hat{y}\rangle\vert \leq \sqrt{d}\Delta + d\frac{\Delta^2}{4}.
$$

In [5]:
def pnorm(x: list[float], p: int = 2) -> float:
    return math.pow(sum(np.abs(x_i) ** p for x_i in x), 1 / p)

def normalize(x: list[float]) -> list[float]:
    norm = pnorm(x, 2)
    return [x_i / norm for x_i in x]

x = normalize([random.random() for _ in range(d)])
y = normalize([random.random() for _ in range(d)])

x_bar = [forward_transformation(x_i) for x_i in x]
y_bar = [forward_transformation(y_i) for y_i in y]

x_hat = [backward_transformation(x_i) for x_i in x_bar]
y_hat = [backward_transformation(y_i) for y_i in y_bar]

dot_product = np.dot(x, y)
dot_product_hat = np.dot(x_hat, y_hat)

error_bound = 2 * delta * math.sqrt(d) + d * delta ** 2

print("<x,y> =", dot_product)
print("<x_hat,y_hat> = ", dot_product_hat)
print("Difference = ", np.abs(dot_product - dot_product_hat))
print("Error estimation = ", 2 * delta * math.sqrt(d) + d * delta ** 2)

<x,y> = 0.745499388482493
<x_hat,y_hat> =  0.7223836985774703
Difference =  0.023115689905022663
Error estimation =  0.035383150127639915


## Efficient calculation of the dot product
Let $x, y\in\mathbb{R}$. Using the backward transformation, we see that $\hat{x} = (\overline{x} - a)\cdot \Delta + \mathrm{min}$ and $\hat{y} = (\overline{y} - a)\cdot \Delta + \mathrm{min}$. Then we see that
$$
\begin{align*}
    \hat{x}\cdot\hat{y} &= ((\overline{x} - a)\cdot \Delta + \mathrm{min})((\overline{y} - a)\cdot \Delta + \mathrm{min})\\
    &= (\overline{x} - a)(\overline{y} - a)\Delta^2 + \mathrm{min}\cdot\Delta(\overline{x} - a)+ \mathrm{min}(\overline{y} - a) + \mathrm{min}^2 \\
    &= \overline{x}\cdot\overline{y}\cdot\Delta^2 + \mathrm{min}\cdot\Delta(\overline{x} - a) - a\Delta^2\overline{x} + \mathrm{min}\cdot\Delta(\overline{y} - a) - a\Delta^2\overline{y} + a^2\Delta^2 + \mathrm{min}^2.
\end{align*}
$$

Now, we consider vectors $x, y\in\mathbb{R}^d$ with quantizations $\overline{x}, \overline{y}$ and reconstrunctions $\hat{x}, \hat{y}$. Then we can calculate the dot product of the reconstructed values as
$$
\begin{align*}
\langle \hat{x}, \hat{y}\rangle &= \sum_{i=1}^d \hat{x}_i\cdot\hat{y}_i\\
&= \sum_{i=1}^d(\overline{x}_i\cdot\overline{y}_i\cdot\Delta^2 + \mathrm{min}\cdot\Delta(\overline{x}_i - a) - a\Delta^2\overline{x}_i + \mathrm{min}\cdot\Delta(\overline{y}_i - a) - a\Delta^2\overline{y}_i + a^2\Delta^2 + \mathrm{min}^2)\\
&= \Delta^2\langle \overline{x}, \overline{y}\rangle + \sum_{i=1}^d(\mathrm{min}\cdot\Delta(\overline{x}_i - a) - a\Delta^2\overline{x}_i) + \sum_{i=1}^d(\mathrm{min}\cdot\Delta(\overline{y}_i - a) - a\Delta^2\overline{y}_i) + d(a^2\Delta^2 + \mathrm{min}^2).
\end{align*}
$$

The interesting fact is, that $d(a^2\Delta^2 + \mathrm{min}^2)$ is independent of $x$ and $y$. Thus, this term only needs to be calculated once for the whole index. Further more, the term $sum_{i=1}^d(\mathrm{min}\cdot\Delta(\overline{x}_i - a) - a\Delta^2\overline{x}_i)$ is only dependent on $x$. Therefore, we can calculate this term when the value is stored in the index.

During inference time, we only need to calculate $\langle\overline{x}, \overline{y}\rangle$, which can be done much more efficiently.

In [6]:
integer_dot_product = sum(x_i * y_i for (x_i, y_i) in zip(x_bar, y_bar))

dot_product_x_part = min_v * delta * (sum(x_i for x_i in x_bar) - d * a) - a * delta ** 2 * sum(x_i for x_i in x_bar)
dot_product_y_part = min_v * delta * (sum(y_i for y_i in y_bar) - d * a) - a * delta ** 2 * sum(y_i for y_i in y_bar)

dot_product_index_part = d * (a ** 2 * delta ** 2 + min_v ** 2)

dot_product_hat_improved = delta ** 2 * integer_dot_product + dot_product_x_part + dot_product_y_part + dot_product_index_part

print("Improved calculation: ", dot_product_hat_improved)

Improved calculation:  0.7223836985774685
